In [ ]:
import pandas as pd
import time
pd.options.mode.chained_assignment = None

# functions
def get_electorate():
    ''' select electorate from text menu '''       
    valid_response = False
    while not valid_response:
        try:
            print("\nWhich electorate do you wish to analyse?")
            for i, electorate in enumerate(electorates.electorate):
                print(f"{i + 1}. {electorate}")
            print("6. quit program")
            text = input("Enter a number from 1 to 6:")            
            text = int(text)
            if (text > 0) and (text < 7):
                valid_response = True
                return text
            else:
                print("That's not a valid choice.")
        except:
            print("That's not a valid choice.")

def eliminate_candidate():
    ''' remove eliminated candidate and return ID '''
    candidate = candidates.index[len(candidates) - 1]
    name = candidates.at[candidate, "cname"]
    party = candidates.at[candidate, "party"]
    primary = candidates.at[candidate, "primary"]
    eliminated.loc[candidate] = [name, party, primary]
    print(f"{name} is eliminated!")
    return candidate

def elect_candidate(candidate):
    ''' declare elected candidate and calculate fractional vote '''
    global elected
    name = candidates.at[candidate, "cname"]
    print(f"{name} is elected!")
    party = candidates.at[candidate, "party"]
    primary = candidates.at[candidate, "primary"]
    elected.loc[candidate] = [name, party, primary]
    # calculate fractional vote    
    fraction = (candidates.at[candidate, "votes"] - quota) / candidates.at[candidate, "votes"]
    print(f"{name} had {candidates.at[candidate, 'votes']:,.1f} votes. Fractional transfer value is {round(fraction, 6)}")
    return fraction

def distribute_votes(candidate, elected):
    ''' redistributes votes/surplus votes from eliminated/elected candidates '''
    global candidates, sample, exhausted
    print(f"redistributing {candidates.at[candidate, 'cname']}'s {'surplus ' if elected else ''}votes ...", end="\r")
    start = time.time()
    interval = 1
    elapsed = interval
    sample_length = len(sample)
    for i, indice in enumerate(sample.index):
        # is vote for this candidate?
        if sample.at[indice, "votes"][sample.at[indice, "pref"]] == candidate:
            if elected:
                sample.at[indice, "value"] = round(sample.at[indice, "value"] * fraction, 6)
            # remove vote from eliminate/elected candidate
            candidates.at[candidate, "votes"] = candidates.at[candidate, "votes"] - sample.at[indice, "value"]
            # is there a next preference?
            preference_resolved = False
            while not preference_resolved:
                if len(sample.at[indice, "votes"]) > sample.at[indice, "pref"] + 1:
                    pref = sample.at[indice, "pref"] + 1
                    sample.at[indice, "pref"] = pref
                    votes = sample.at[indice, "votes"]
                    value = sample.at[indice, "value"]
                    # is preferenced candidate still in count?
                    if votes[pref] in candidates.index:
                        # add vote to preferenced candidate
                        candidates.at[votes[pref], "votes"] = candidates.at[votes[pref], "votes"] + value
                # add to exhausted votes
                else:                
                    exhausted = exhausted + sample.at[indice, "value"]
                    sample.drop(index=indice, inplace=True)
                    preference_resolved = True
        if (time.time() - elapsed) > start:
            print(f"redistributing {candidates.at[candidate, 'cname']}'s {'surplus ' if elected else ''}votes ... {(i + 1) / sample_length:.1%}", end="\r")
            elapsed = elapsed + interval
    print(f"redistributing {candidates.at[candidate, 'cname']}'s {'surplus ' if elected else ''}votes ... complete")

def record_count(event):
    ''' records current vote tally and saves to file '''
    global results_phase
    results_phase = results_phase + 1    
    results.at[results_phase, "event"] = event
    results.at[results_phase, "exhausted"] = exhausted
    for candidate in elected.index:
        results.at[count_no, elected.at[candidate, "cname"]] = "elected"    
    for candidate in eliminated.index:
        results.at[count_no, eliminated.at[candidate, "cname"]] = "eliminated"
    for candidate in candidates.index:
        results.at[count_no, candidates.at[candidate, "cname"]] = candidates.at[candidate, "votes"]
    results.to_csv(f"./data/results_{electorates.at[electorate, 'electorate']}.csv")

# BEGIN PROGRAM

print("2020 ACT ELECTION SIMULATOR")
print("\u00a9 Markus Mannheim (ABC Canberra)")

# read in electorates
electorates = pd.read_csv("./data/Electorates.txt", index_col="ecode")

# begin cycle
while True:
    electorate = get_electorate()
    
    # user wants to quit
    if electorate == 6:
        break
    
    # load electorate data
    print(f"\nloading {electorates.at[electorate, 'electorate']} data ...", end="\r")
    candidates = pd.read_csv(f"./data/candidates_{electorates.at[electorate, 'electorate']}.csv", index_col="id")
    parties = pd.read_csv(f"./data/parties_{electorates.at[electorate, 'electorate']}.csv", index_col="id")
    votes = pd.read_csv(f"./data/votes_{electorates.at[electorate, 'electorate']}.csv", index_col="id")
    votes.votes = votes.votes.apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(", "))
    print(f"loading {electorates.at[electorate, 'electorate']} data ... complete")
    
    # create sample data to speed up calculation
    print("sampling data for analysis ...", end="\r")
    sample_size = .1
    sample = votes.sample(frac=sample_size)
    print("sampling data for analysis ... complete")
 
    # prepare datasets to contain elected and eliminated candidates
    print("preparing results containers ...", end="\r")
    elected = pd.DataFrame(columns=["cname", "party", "primary"])
    eliminated = pd.DataFrame(columns=["cname", "party", "primary"])
    exhausted = 0.0
    results = pd.DataFrame(columns=list(candidates.cname) + ["exhausted", "event"])
    results.index.name = "phase"
    print("preparing results containers ... complete")

    # establish quota
    total_votes = len(sample)
    quota = int(total_votes / 6) + 1
    print(f"\nquota for {electorates.at[electorate, 'electorate']}: {format(quota, ',')} votes")

    # record primary votes
    print("recording primary votes ...", end="\r")
    start = time.time()
    interval = 1
    elapsed = interval
    for i, indice in enumerate(sample.index):
        candidate = sample.at[indice, "votes"][0]
        candidates.at[candidate, "votes"] = candidates.at[candidate, "votes"] + 1
        if (time.time() - elapsed) > start:
            print(f"recording primary votes ... {(i + 1) / len(sample):.1%}", end="\r")
    candidates.primary = candidates.votes.apply(lambda x: f"{x / total_votes:.1%}")
    candidates.sort_values("votes", ascending=False, inplace=True)
    print("recording primary votes ... complete")
    
    # begin counting cycle
    count_no = 1
    results_phase = 0
    record_count("primary votes")
    while len(elected) < 5:
        print(f"\nCOUNT No. {count_no}\n")
        # check if candidates elected
        reached_quota = candidates[candidates["votes"] >= quota]
        if len(reached_quota) == 0:
            print("no candidates elected")
            eliminated_candidate = eliminate_candidate()
            distribute_votes(eliminated_candidate, False)
            record_count(f"{eliminated.at[eliminated_candidate, 'cname']} eliminated")
            candidates.drop(index=eliminated_candidate, inplace=True)
        else:
            i = 0
            while len(elected) < 5:
                candidate = reached_quota.index[i]
                fraction = elect_candidate(candidate)
                distribute_votes(candidate, True)
                record_count(f"{elected.at[candidate, 'cname']} elected")
                candidates.drop(index=candidate, inplace=True)
                i = i + 1
                if len(reached_quota) < i + 1:
                    break
        candidates.sort_values("votes", ascending=False, inplace=True)
        count_no = count_no + 1

    print()
    print(pd.concat([elected, candidates, eliminated.iloc[::-1]]))
    print(f"exhausted votes: {exhausted:,.1}")

# exit program
print("\nEnjoy your day.")

2020 ACT ELECTION SIMULATOR
© Markus Mannheim (ABC Canberra)

Which electorate do you wish to analyse?
1. Brindabella
2. Ginninderra
3. Kurrajong
4. Murrumbidgee
5. Yerrabi
6. quit program


Enter a number from 1 to 6: 1



loading Brindabella data ... complete
sampling data for analysis ... complete
preparing results containers ... complete

quota for Brindabella: 914 votes
recording primary votes ... complete

COUNT No. 1

no candidates elected
Scott SANDFORD is eliminated!
redistributing Scott SANDFORD's votes ... complete

COUNT No. 2

no candidates elected
Matthew KNIGHT is eliminated!
redistributing Matthew KNIGHT's votes ... complete

COUNT No. 3

no candidates elected
Jacob GOWOR is eliminated!
redistributing Jacob GOWOR's votes ... complete

COUNT No. 4

no candidates elected
Robyn SOXSMITH is eliminated!
redistributing Robyn SOXSMITH's votes ... complete

COUNT No. 5

no candidates elected
Jason POTTER is eliminated!
redistributing Jason POTTER's votes ... complete

COUNT No. 6

no candidates elected
Jannah FAHIZ is eliminated!
redistributing Jannah FAHIZ's votes ... complete

COUNT No. 7

no candidates elected
Adrian OLLEY is eliminated!
redistributing Adrian OLLEY's votes ... complete

COUNT 